# Temp

In [ ]:
# Temp

In [66]:
import cv2
def draw_boxes_for_label_zero(file_path, image_path, image_width, image_height, output_path):
    # Đọc ảnh
    image = cv2.imread(image_path)

    # Mở file chứa thông tin bounding box
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Lặp qua từng dòng trong file
    for line in lines:
        # Tách các giá trị trong dòng
        values = line.strip().split()
        label = int(values[0])
        x_center = float(values[1])
        y_center = float(values[2])
        width = float(values[3])
        height = float(values[4])
        
        # Kiểm tra nếu label là 0
        if label == 0:
            # Chuyển đổi từ các giá trị center và kích thước (relative to image size) sang pixel
            x1 = int((x_center - width / 2) * image_width)
            y1 = int((y_center - height / 2) * image_height)
            x2 = int((x_center + width / 2) * image_width)
            y2 = int((y_center + height / 2) * image_height)

            # Vẽ bounding box lên ảnh
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Màu xanh lá, độ dày 2

    # Lưu ảnh đã vẽ
    cv2.imwrite(output_path, image)
    print(f"Image saved at {output_path}")

# Gọi hàm
file_path = "IMG_1581_iter_0.txt"  # Đường dẫn tới file label
image_path = "IMG_1581_iter_0.jpg"  # Đường dẫn tới ảnh gốc
output_path = "avg_coords.jpg"  # Đường dẫn lưu ảnh đầu ra
image = cv2.imread(image_path)
h, w = image.shape[:2]

draw_boxes_for_label_zero(file_path, image_path, w, h, output_path)


Image saved at avg_coords.jpg


In [65]:
import cv2
import random
from train_utils import *
import matplotlib.pyplot as plt

inputs = []
targets = []
from fixed_coordinates import fixed_circle
image = cv2.imread("IMG_1581_iter_0.jpg")
img_path = "IMG_1581_iter_0.jpg"
coord_saver = "test.txt"
fixed_circle(img_path, coord_saver)
labels, coords = read_file_to_tensors("test.txt")
for i, coord in enumerate(coords):
    x1, y1, x2, y2 = find_yolov8_square(image, coord)
    rand1 = random.randint(-10, 10)
    rand2 = random.randint(-10, 10)
    x1 = x1 + rand1
    x2 = x2 + rand1
    y1 = y1 + rand2
    y2 = y2 + rand2
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
cv2.imwrite("/Users/phamminhtuan/Desktop/AIChallenge/avg_coords.jpg", image)

Centers of black squares (YOLOv8 format): [(0.9157427937915743, 0.928911456680419), (0.7707317073170732, 0.9247857822913361), (0.6283813747228382, 0.9216121866074262), (0.4878048780487805, 0.9190733100602984), (0.34767184035476717, 0.9174865122183434), (0.20709534368070953, 0.9165344335131704), (0.06607538802660753, 0.9152649952396065), (0.9152993348115299, 0.6708981275785465), (0.7725055432372505, 0.6686766105998095), (0.7015521064301552, 0.6677245318946367), (0.6310421286031042, 0.6667724531894637), (0.4909090909090909, 0.6651856553475087), (0.3507760532150776, 0.6642335766423357), (0.2811529933481153, 0.6635988575055538), (0.2106430155210643, 0.6632814979371628), (0.06873614190687362, 0.6620120596635989), (0.9170731707317074, 0.5445890193589337), (0.7033259423503326, 0.5420501428118057), (0.49223946784922396, 0.5401459854014599), (0.282039911308204, 0.5385591875595049), (0.06962305986696231, 0.536337670580768), (0.9206208425720621, 0.34306569343065696), (0.8310421286031042, 0.326880

True

# Dataset

In [7]:
import torch
import random
from torchvision import transforms
import cv2
from train_utils import *
class MyDataset(torch.utils.data.Dataset):
    """
    Initializes the dataset by reading the image and label files, and processes them into tensors.

    Args:
        image_path (str): Path to the input image.
        label_path (str): Path to the label file (YOLO format).

    The method reads the image from the specified path and the label data, then processes each labeled 
    bounding box to crop and transform the image into a tensor. The processed tensors and corresponding 
    labels are stored in `self.inputs` and `self.targets` respectively.
    """
    def __init__(self, image_path, label_path):
        super().__init__()
        self.image_path = image_path
        self.label_path = label_path
        self.inputs, self.targets = [], []
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((32, 32))
        ])
        image = cv2.imread(self.image_path)
        labels, coords = read_file_to_tensors(self.label_path)
        for i, coord in enumerate(coords):
            rand1 = random.randint(-10, 10)
            rand2 = random.randint(-10, 10)
            coord[0] += rand1/2255
            coord[1] += rand2/3151
            square = find_yolov8_square(image, coord)
            cropped_image = get_box(image, square)
            cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
            cropped_tensor = self.transform(cropped_image)
            self.inputs.append(cropped_tensor)
            self.targets.append(labels[i])
    def __getitem__(self, idx):
        image = self.inputs[idx]
        label = self.targets[idx]
        return image, label
    def __len__(self):
        return len(self.inputs)

In [44]:
import os
image_folder = "/Users/phamminhtuan/Downloads/Trainning_SET/Images"
label_folder = "/Users/phamminhtuan/Downloads/Trainning_SET/Labels"
train = []

for filename in os.listdir(image_folder):
    if filename.endswith('.jpg') and 'iter_' in filename:
        # Lấy số từ tên file
        try:
            number = int(filename.split('iter_')[1].split('.jpg')[0])
            if 0 <= number <= 15:  # Kiểm tra nếu số nằm trong khoảng 0-10
                # Lấy đường dẫn file ảnh và nhãn
                image_path = os.path.join(image_folder, filename)
                label_path = os.path.join(label_folder, os.path.splitext(filename)[0] + ".txt")
                # Tạo dataset tạm thời
                temp = MyDataset(image_path, label_path)
                # Thêm các phần tử từ dataset vào train
                train.extend(temp[i] for i in range(len(temp)))
        except ValueError:
            # Bỏ qua file nếu không trích xuất được số hợp lệ
            pass




In [49]:
len(train)

171802

In [63]:
import os
image_folder = "/Users/phamminhtuan/Downloads/Trainning_SET/Images"
label_folder = "/Users/phamminhtuan/Downloads/Trainning_SET/Labels"
test = []
for filename in os.listdir(image_folder):
    if filename.endswith('.jpg') and 'iter_' in filename:
        # Lấy số từ tên file
        try:
            number = int(filename.split('iter_')[1].split('.jpg')[0])
            if 16 <= number <= 18:  # Kiểm tra nếu số nằm trong khoảng 0-10
                # Lấy đường dẫn file ảnh và nhãn
                image_path = os.path.join(image_folder, filename)
                label_path = os.path.join(label_folder, os.path.splitext(filename)[0] + ".txt")
                # Tạo dataset tạm thời
                temp_test = MyDataset(image_path, label_path)
                # Thêm các phần tử từ dataset vào train
                test.extend(temp_test[i] for i in range(len(temp_test)))
        except ValueError:
            # Bỏ qua file nếu không trích xuất được số hợp lệ
            pass

            

In [74]:
print(len(train), len(test))
print(temp[0][0].shape,temp[0][1].shape, test[0][0].shape)


171802 33391
torch.Size([3, 32, 32]) torch.Size([]) torch.Size([3, 32, 32])


In [73]:
train_loader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True)
val_loader = torch.utils.data.DataLoader(test, batch_size=60, shuffle=True)
print (len(train_loader), len(val_loader))

1343 557


# Model

## CNN

In [74]:
import torch.nn as nn
class EfficientCNN(nn.Module):
    def __init__(self):
        """
        Initializes the EfficientCNN model. This model is an efficient convolutional neural network for the 
        classification task. It consists of a feature extraction part and a classification part. The feature 
        extraction part is a convolutional neural network which consists of 3 convolutional layers with 
        maxpooling, where the number of channels are 32, 64, 128 respectively. The classification part is a 
        fully connected neural network which consists of 2 fully connected layers with dropout rate 0.5.

        Args:
            None
        """
        super(EfficientCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, 2),
            
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        x = torch.sigmoid(x)
        return x
model = EfficientCNN()
import torchsummary
torchsummary.summary(model,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
         MaxPool2d-4           [-1, 32, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]          18,496
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
         MaxPool2d-8             [-1, 64, 8, 8]               0
            Conv2d-9            [-1, 128, 8, 8]          73,856
      BatchNorm2d-10            [-1, 128, 8, 8]             256
             ReLU-11            [-1, 128, 8, 8]               0
        MaxPool2d-12            [-1, 128, 4, 4]               0
           Linear-13                  [-1, 256]         524,544
             ReLU-14                  [

## MobileNet

In [67]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(DepthwiseSeparableConv, self).__init__()
        # Depthwise convolution
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels)

        # Pointwise convolution
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = torch.nn.functional.relu6(self.bn1(self.depthwise(x)))
        x = torch.nn.functional.relu6(self.bn2(self.pointwise(x)))
        return x

In [130]:
class MobileNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(MobileNet, self).__init__()

        def conv_bn(in_channels, out_channels, stride):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 3, stride, 1, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU6(inplace=True)
            )

        self.model = nn.Sequential(
            conv_bn(3, 32, 2),  # input size: 32x32x3, output size: 16x16x32
            DepthwiseSeparableConv(32, 64, 1),  # output size: 16x16x64
            DepthwiseSeparableConv(64, 128, 2), # output size: 8x8x128
            DepthwiseSeparableConv(128, 128, 1),# output size: 8x8x128
            DepthwiseSeparableConv(128, 256, 2),# output size: 4x4x256
            DepthwiseSeparableConv(256, 256, 1),# output size: 4x4x256
            DepthwiseSeparableConv(256, 512, 2),# output size: 2x2x512
            DepthwiseSeparableConv(512, 512, 1),# 5x repeated, output size: 2x2x512
            DepthwiseSeparableConv(512, 512, 1),
            DepthwiseSeparableConv(512, 512, 1),
            DepthwiseSeparableConv(512, 512, 1),
            DepthwiseSeparableConv(512, 512, 1),
            DepthwiseSeparableConv(512, 1024, 2),# output size: 1x1x1024
            DepthwiseSeparableConv(1024, 1024, 1),# output size: 1x1x1024
            nn.AdaptiveAvgPool2d(1), # output size: 1x1x1024
        )
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = torch.sigmoid(x)
        return x

# Khởi tạo mô hình
model = MobileNet(num_classes=2)  # Chuyển mô hình sang GPU nếu có

# Tóm tắt mô hình
torchsummary.summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 16, 16]             864
       BatchNorm2d-2           [-1, 32, 16, 16]              64
             ReLU6-3           [-1, 32, 16, 16]               0
            Conv2d-4           [-1, 32, 16, 16]             288
       BatchNorm2d-5           [-1, 32, 16, 16]              64
            Conv2d-6           [-1, 64, 16, 16]           2,048
       BatchNorm2d-7           [-1, 64, 16, 16]             128
DepthwiseSeparableConv-8           [-1, 64, 16, 16]               0
            Conv2d-9             [-1, 64, 8, 8]             576
      BatchNorm2d-10             [-1, 64, 8, 8]             128
           Conv2d-11            [-1, 128, 8, 8]           8,192
      BatchNorm2d-12            [-1, 128, 8, 8]             256
DepthwiseSeparableConv-13            [-1, 128, 8, 8]               0
           Conv2d-14          

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.models import VisionTransformer as ViT

# Khởi tạo mô hình, loss function và optimizer
model = ViT(
            image_size=32,  # Kích thước ảnh đầu vào là 32x32
            patch_size=8,   # Kích thước patch sẽ giảm xuống (kích thước này là tùy chọn)
            num_classes=2,
            hidden_dim=256,        # Kích thước đầu ra của Transformer Encoder (có thể tùy chỉnh)
            num_layers=6,        # Số lớp của Transformer Encoder
            num_heads=8,        # Số attention heads
            mlp_dim=512     # Kích thước của MLP sau attention layers
        )
# Kiểm tra thông tin mô hình (tùy chọn)
# torchsummary.summary(model, (3, 32, 32))  # Đảm bảo input có kích thước (3, 32, 32)



KeyboardInterrupt: 

# Train

In [66]:
device = torch.device("mps")

# Train 2

In [29]:
import pytorch_lightning as pl

In [31]:
class CroatianFishClassifier(pl.LightningModule):
    def __init__(self, num_classes):
        super(CroatianFishClassifier, self).__init__()
        self.model = model
        self.train_accuracies = []

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        logits = self(images)
        loss = torch.nn.functional.cross_entropy(logits, labels)
        acc = self.accuracy(logits, labels)
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        logits = self(images)
        loss = torch.nn.functional.cross_entropy(logits, labels)
        acc = self.accuracy(logits, labels)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=5e-4)

    def accuracy(self, logits, labels):
        _, predicted = torch.max(logits.data, 1)
        correct = (predicted == labels).sum().item()
        total = labels.size(0)
        out = correct / total
        return out

    def test_step(self, batch, batch_idx):
        images, labels = batch
        logits = self(images)
        loss = F.cross_entropy(logits, labels)
        acc = self.accuracy(logits, labels)
        self.log('test_loss', loss, prog_bar=True)
        self.log('test_acc', acc, prog_bar=True)
        return {'test_loss': loss, 'test_acc': acc}

In [75]:
# Định nghĩa ModelCheckpoint callback
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor='val_acc',
    filename='best-checkpoint',
    save_top_k=1,
    mode='max',
    verbose=True
)

# Định nghĩa progress bar callback
progress_bar = pl.callbacks.TQDMProgressBar()

# Các tham số Trainer
PARAMS = {
    "benchmark": True,
    "enable_progress_bar": True,
    "logger": True,
    "callbacks": [progress_bar, checkpoint_callback],
    "log_every_n_steps": 1,
    "num_sanity_val_steps": 0,
    "max_epochs": 30,
    "precision": 16
}

# Kiểm tra xem có checkpoint tốt nhất không
best_checkpoint_path = checkpoint_callback.best_model_path
if best_checkpoint_path:
    print(f"Resuming training from best checkpoint: {best_checkpoint_path}")
    PARAMS["resume_from_checkpoint"] = best_checkpoint_path

# Khởi tạo Trainer
trainer = pl.Trainer(**PARAMS, accelerator='mps', devices=1)

# Khởi tạo model
Train = CroatianFishClassifier(num_classes=2)

# Huấn luyện model
trainer.fit(Train, train_loader, val_loader)


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type              | Params
--------------------------------------------
0 | model | VisionTransformer | 3.2 M 
--------------------------------------------
3.2 M     Trainable params
0         Non-trainable params
3.2 M     Total params
12.871    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 0, global step 1343: 'val_acc' reached 0.97098 (best 0.97098), saving model to '/Users/phamminhtuan/Desktop/AIChallenge/lightning_logs/version_15/checkpoints/best-checkpoint.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

...

Epoch 4, global step 6715: 'val_acc' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 5, global step 8058: 'val_acc' reached 0.97242 (best 0.97242), saving model to '/Users/phamminhtuan/Desktop/AIChallenge/lightning_logs/version_15/checkpoints/best-checkpoint.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 6, global step 9401: 'val_acc' reached 0.97254 (best 0.97254), saving model to '/Users/phamminhtuan/Desktop/AIChallenge/lightning_logs/version_15/checkpoints/best-checkpoint.ckpt' as top 1
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


# Data augment

In [19]:
import matplotlib.pyplot as plt

In [20]:
def remove_duplicate_lines(file_path):
    try:
        # Đọc nội dung file và loại bỏ các dòng trùng lặp
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            unique_lines = list(dict.fromkeys(lines))  # Loại bỏ các dòng trùng lặp và giữ nguyên thứ tự
        
        # Ghi lại nội dung không trùng lặp vào file
        with open(file_path, 'w', encoding='utf-8') as file:
            file.writelines(unique_lines)
        print("Đã loại bỏ các dòng trùng lặp thành công.")
    except FileNotFoundError:
        print(f"File {file_path} không tồn tại.")
    except Exception as e:
        print(f"Đã xảy ra lỗi: {e}")

In [21]:
def remove_duplicates_column2_and_column3(file_path, output_path):
    try:
        # Đọc file và lưu dữ liệu
        with open(file_path, 'r', encoding='utf-8') as infile:
            lines = infile.readlines()

        # Tách các dòng thành danh sách các cột
        rows = [line.strip().split() for line in lines]

        # Lưu tần suất xuất hiện của các giá trị ở cột 2 và cột 3
        counts = {}
        for row in rows:
            if len(row) > 2:  # Đảm bảo có ít nhất 3 cột
                key = (row[1], row[2])  # Chỉ lấy giá trị ở cột 2 và cột 3
                counts[key] = counts.get(key, 0) + 1

        # Lọc bỏ các dòng có giá trị ở cột 2 và cột 3 trùng lặp
        cleaned_rows = [
            row for row in rows
            if len(row) <= 2 or counts.get((row[1], row[2]), 0) == 1
        ]

        # Ghi lại các dòng không trùng lặp vào file đầu ra
        with open(output_path, 'w', encoding='utf-8') as outfile:
            for row in cleaned_rows:
                outfile.write(' '.join(row) + '\n')  # Sử dụng khoảng trắng để ngăn cách các cột

        print("Đã loại bỏ các dòng trùng lặp ở cột 2 và cột 3.")
    except FileNotFoundError:
        print(f"File {file_path} không tồn tại.")
    except Exception as e:
        print(f"Đã xảy ra lỗi: {e}")

In [1]:
import os

def remove_duplicate_lines_by_columns(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    unique_lines = []
    seen = set()

    for line in lines:
        columns = line.strip().split()  # Giả sử các cột được phân cách bằng khoảng trắng
        if len(columns) >= 3:  # Kiểm tra nếu dòng có ít nhất 3 cột
            column2_3 = (columns[1], columns[2])  # Lấy giá trị ở cột 2 và cột 3
            if column2_3 not in seen:
                unique_lines.append(line)
                seen.add(column2_3)
    
    # Ghi lại vào file
    with open(file_path, 'w') as file:
        file.writelines(unique_lines)

def process_txt_files_in_directory(directory_path):
    # Duyệt qua tất cả các file trong thư mục
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):  # Kiểm tra file .txt
            file_path = os.path.join(directory_path, filename)
            remove_duplicate_lines_by_columns(file_path)
            print(f"Processed: {file_path}")

# Đường dẫn đến thư mục chứa các file .txt
directory_path = "/Users/phamminhtuan/Desktop/Trainning_SET/Labels"

# Gọi hàm để xử lý tất cả các file .txt trong thư mục
process_txt_files_in_directory(directory_path)


Processed: /Users/phamminhtuan/Desktop/Trainning_SET/Labels/IMG_1590_iter_248.txt
Processed: /Users/phamminhtuan/Desktop/Trainning_SET/Labels/IMG_2274_iter_248.txt
Processed: /Users/phamminhtuan/Desktop/Trainning_SET/Labels/IMG_1591_iter_98.txt
Processed: /Users/phamminhtuan/Desktop/Trainning_SET/Labels/IMG_1591_iter_139.txt
Processed: /Users/phamminhtuan/Desktop/Trainning_SET/Labels/IMG_2271_iter_46.txt
Processed: /Users/phamminhtuan/Desktop/Trainning_SET/Labels/IMG_1591_iter_111.txt
Processed: /Users/phamminhtuan/Desktop/Trainning_SET/Labels/IMG_1587_iter_16.txt
Processed: /Users/phamminhtuan/Desktop/Trainning_SET/Labels/IMG_3955_iter_7.txt
Processed: /Users/phamminhtuan/Desktop/Trainning_SET/Labels/IMG_1591_iter_105.txt
Processed: /Users/phamminhtuan/Desktop/Trainning_SET/Labels/IMG_2271_iter_52.txt
Processed: /Users/phamminhtuan/Desktop/Trainning_SET/Labels/IMG_2266_iter_0.txt
Processed: /Users/phamminhtuan/Desktop/Trainning_SET/Labels/IMG_2270_iter_240.txt
Processed: /Users/phammi

In [2]:
import os

def count_lines_in_txt_files(directory_path):
    # Duyệt qua tất cả các file trong thư mục
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):  # Kiểm tra file .txt
            file_path = os.path.join(directory_path, filename)
            
            # Đếm số dòng trong file
            try:
                with open(file_path, 'r') as file:
                    lines = file.readlines()
                    print(f"File: {filename}, Số dòng: {len(lines)}")
            except Exception as e:
                print(f"Không thể đọc file {filename}: {e}")

# Đường dẫn đến thư mục chứa các file .txt
directory_path = "/Users/phamminhtuan/Desktop/Trainning_SET/Labels"

# Gọi hàm để đếm số dòng của tất cả các file .txt trong thư mục
count_lines_in_txt_files(directory_path)


File: IMG_1590_iter_248.txt, Số dòng: 573
File: IMG_2274_iter_248.txt, Số dòng: 572
File: IMG_1591_iter_98.txt, Số dòng: 572
File: IMG_1591_iter_139.txt, Số dòng: 572
File: IMG_2271_iter_46.txt, Số dòng: 572
File: IMG_1591_iter_111.txt, Số dòng: 572
File: IMG_1587_iter_16.txt, Số dòng: 572
File: IMG_3955_iter_7.txt, Số dòng: 572
File: IMG_1591_iter_105.txt, Số dòng: 572
File: IMG_2271_iter_52.txt, Số dòng: 572
File: IMG_2266_iter_0.txt, Số dòng: 572
File: IMG_2270_iter_240.txt, Số dòng: 573
File: IMG_1581_iter_137.txt, Số dòng: 571
File: IMG_2265_iter_123.txt, Số dòng: 572
File: IMG_2271_iter_125.txt, Số dòng: 572
File: IMG_3955_iter_256.txt, Số dòng: 572
File: IMG_2271_iter_131.txt, Số dòng: 572
File: IMG_3955_iter_242.txt, Số dòng: 572
File: IMG_2265_iter_137.txt, Số dòng: 572
File: IMG_2270_iter_254.txt, Số dòng: 573
File: IMG_1581_iter_123.txt, Số dòng: 571
File: IMG_2271_iter_85.txt, Số dòng: 572
File: IMG_2267_iter_23.txt, Số dòng: 572
File: IMG_1591_iter_73.txt, Số dòng: 572
Fil

# Test lại

In [18]:
import torch
import torch.nn as nn

def test_model(model, val_loader, device='cpu'):
    """
    Test the given model on the test dataset.

    Args:
        model (nn.Module): The trained model to evaluate.
        test_loader (DataLoader): A DataLoader for the test data.
        device (str, optional): The device to use for testing ('cpu' or 'cuda'). Defaults to 'cpu'.

    Returns:
        float: Test accuracy (in percentage).
        float: Test loss.
    """
    model = model.to(device)
    model.eval()  # Chuyển sang chế độ đánh giá
    criterion = nn.CrossEntropyLoss()

    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # Tắt tính toán gradient để tăng tốc
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            labels = labels.long()
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = 100 * correct / total
    average_test_loss = test_loss / len(val_loader)

    print(f'Test Loss: {average_test_loss:.4f}')
    print(f'Test Accuracy: {test_accuracy:.2f}%')

    return test_accuracy, average_test_loss


In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = EfficientCNN()
model.load_state_dict(torch.load('/Users/phamminhtuan/Desktop/AIChallenge/cnn.pth'))
model = model.to(device)

# Đánh giá mô hình
test_accuracy, test_loss = test_model(model, val_loader, device=device)

/var/folders/hv/wh1bzsb91mz5swh0znhrdrv00000gn/T/ipykernel_45970/1814793565.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/Users/pham

Test Loss: 0.0046
Test Accuracy: 99.91%


In [20]:
image_path = "/Users/phamminhtuan/Desktop/Trainning_SET/Images/IMG_1587_iter_12.jpg"
label_path = "/Users/phamminhtuan/Desktop/Trainning_SET/Labels/IMG_1587_iter_12.txt"


In [21]:
import utils
from torchvision import transforms 
image = cv2.imread(image_path)
labels, coords = read_file_to_tensors(label_path)
for i, coord in enumerate(coords):
    xy_coord = find_yolov8_square(image, coord)
    x1, y1, x2, y2 = xy_coord
    input = get_box(image, xy_coord)
    transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((32, 32))
        ])
    input = transform(input)
    output = model(input.unsqueeze(0))
    
cv2.imwrite("/Users/phamminhtuan/Desktop/AIChallenge/avg_coords.jpg", image)

True

In [3]:
from train_utils import *
folder_path = "/Users/phamminhtuan/Downloads/testset1/images"
prefixes = get_prefixes_with_two_underscores(folder_path)
for prefix in prefixes:
    print(prefix)

IMG_3974
IMG_3968
IMG_3976
IMG_3967
IMG_3975
IMG_3960
IMG_3973
IMG_3966
z6020062307339_f2abaff0550ac8af6e8e586aacd473c8


In [ ]:
from train_utils import *
folder_path = "/Users/phamminhtuan/Downloads/testset1/images"
prefixes = get_prefixes_with_two_underscores(folder_path)
for prefix in prefixes:
    print(prefix)

IMG_3974
IMG_3968
IMG_3976
IMG_3967
IMG_3975
IMG_3960
IMG_3973
IMG_3966
z6020062307339_f2abaff0550ac8af6e8e586aacd473c8


ModuleNotFoundError: No module named 'result_processing'